In [1]:
import time
from duckduckgo_search import DDGS
from sklearn.metrics.pairwise import cosine_similarity
import json
from bs4 import BeautifulSoup
import requests
from openai import OpenAI
from langchain_text_splitters import MarkdownHeaderTextSplitter
import markdownify 

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('keepitreal/vietnamese-sbert')

c:\Users\My_Pc\anaconda3\envs\Minh-AI\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
import os

os.environ["OPENAI_API_KEY"] = "sk-None-Y8DoDJRGH63XFz1edTyaT3BlbkFJMtiwHXQBmx5addWF1b57"

In [4]:
# Hàm để chuyển đổi văn bản thành embedding
def get_embedding(text, model):
    return model.encode(text)

In [5]:
def find_max_result_question(input_text, questions, model):
    if not questions:
        return input_text
    embeddings = [get_embedding(doc.lower(), model) for doc in questions + [input_text.lower()]]
    cosine_similarities = cosine_similarity([embeddings[-1]], embeddings[:-1])
    
    # Tìm câu có độ tương đồng cao nhất
    max_similarity = max(cosine_similarities[0])
    max_index = cosine_similarities[0].tolist().index(max_similarity)
    return max_index

In [6]:
def find_max_result_google_search(input_text, results, model):
    documents = [temp['title'].lower() + ' ' + temp['body'].lower() for temp in results]
    # title
    if documents:
        # Chuyển đổi các văn bản thành embeddings
        embeddings = [get_embedding(doc.lower(), model) for doc in documents + [input_text.lower()]]
        cosine_similarities = cosine_similarity([embeddings[-1]], embeddings[:-1])
        
        # Tìm câu có độ tương đồng cao nhất
        max_similarity = max(cosine_similarities[0])
        max_index = cosine_similarities[0].tolist().index(max_similarity)
        return results[max_index]
    return ''

In [7]:
def google_search(input_text):
    # black_list = '-site:https://thuvienphapluat.vn/cong-dong-dan-luat -site:https://thuvienphapluat.vn/van-ban'
    # black_list = '-site:https://lawnet.vn -site:https://vksndtc.gov.vn -site:thuenhanuoc.vn -site:https://lawnet.vn/vb -site:https://thuvienphapluat.vn/cong-dong-dan-luat -site:psyme.org -site:youtube.com -site:luatminhkhue.vn -site:https://thuvienphapluat.vn/van-ban -site:https://covanphaply.vn/bang -site:https://vietnamnet.vn'
    # + '+site:luatvietnam.vn ' 
    black_list = ' -site:https://vietnamtourism.gov.vn -site:https://thuvienphapluat.vn/van-ban -site:https://thuvienphapluat.vn/cong-dong-dan-luat -site:https://vi.wikipedia.org -site:https://careerviet.vn -site:https://lawkey.vn -site:https://chinhphu.vn/ -site:https://luatminhkhue.vn -site:https://dangkykinhdoanh.gov.vn -site:https://baomoi.com -site:https://gaadvisor.net/ -site:https://www.pace.edu.vn'
    search_query = input_text + black_list
    # #  + ' ' + black_list
    # # search_query = f'site:luatvietnam.vn {input_text} OR site:thuvienphapluat.vn {input_text} {black_list}'
    # 'site:luatvietnam.vn' + ' ' + 
    try:
        results = DDGS().text(
            keywords = search_query,
            region = 'vn-vi',
            safesearch= 'moderate',
            # timelimit= 'y',
            max_results= 5
        )
        # # Biểu thức regex để loại bỏ các URL không mong muốn
        # regex_pattern = r'-d([1-9]|10)\.html'
        # # Lọc kết quả bằng regex
        # filtered_results = [result for result in results if not re.search(regex_pattern, result['href'])]
        return results
    except Exception as e:
        print(f"Rate limit or other exception: {e}")
        # Wait for a random time between 5 to 10 seconds
        time.sleep(21)
        return google_search(input_text)

test

In [8]:
def crawl_data(data):  
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    # Gửi yêu cầu HTTP GET đến URL
    response = requests.get(data.get('href'), headers=headers, verify= False)
    # , verify= False
    # response = requests.get(data.get('href'), headers=headers, verify=False, allow_redirects=True, max_redirects=10)
    soup_data = BeautifulSoup(response.content, 'html5lib', from_encoding= 'utf-8')
    for lead in soup_data.find_all(class_='lead'):
        lead.decompose()
    body_content = soup_data.find('body')
    # Loại bỏ tất cả các phần tử có class="lead"
    # print(body_content)
    return body_content

main

In [9]:
def post_request(url, headers):
    response = requests.post(url, headers=headers)
    return response.json()


def post_request_key(url, headers, data):
    try:
        data = requests.post(url, headers=headers, data=json.dumps(data))
        data.raise_for_status()  # Raise an error for bad status codes
        return data.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None


In [10]:
def get_api_lawid(lawtitle, model):
    url1 = 'https://apids.thuvienphapluat.vn/auth/get-token?key=pvaG4gRG9lIiwiaWF0IjoxNTE2MjM5MDIyfQS563xADXH'
    headers1 = {
        'Cookie': 'Culture=vi; Culture=vi'
    }
    key = post_request(url1, headers1)
    
    payload = ""
    url = f"https://apids.thuvienphapluat.vn/data/search-document?q={lawtitle}"
    headers = {
        'Authorization': 'Bearer ' + key['Data']['AccessToken'],
        'Cookie': 'Culture=vi; Culture=vi; Culture=vi; Culture=vi; Culture=vi',
        'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    result = response.text
    result = json.loads(result)
    lawtitle_list = []
    for temp in result:
        # print(temp['News_Subject'])
        lawtitle_list.append(temp['News_Subject'])
        # print(temp)
        # print('----------------------')
    # print(lawtitle_list[find_max_result_question(lawtitle, lawtitle_list, model)])
    return result[find_max_result_question(lawtitle, lawtitle_list, model)]['LawID']

In [11]:

def generate_JSON_chunking(chunking):
    client = OpenAI()
    few_shot_examples = [
        {"role": "user", "content": "Theo Điều 15, Khoản 2, Điểm a của Luật Giao thông đường bộ, người điều khiển phương tiện phải tuân thủ tín hiệu đèn giao thông."},
        {"role": "assistant", "content": '[{"dieu": 15, "khoan": 2, "diem": "a", "ten_vanban": "Luật Giao thông đường bộ"}]'},
        {"role": "user", "content": "Căn cứ theo Điều 12 Luật Bảo hiểm y tế 2008 (được sửa đổi bởi khoản 6 Điều 1 Luật Bảo hiểm y tế sửa đổi 2014), trẻ em là một trong những đối tượng bắt buộc phải tham gia bảo hiểm y tế. Cụ thể, Luật quy định rõ rằng trẻ em dưới 6 tuổi được cấp thẻ bảo hiểm y tế miễn phí, còn trẻ em từ 6 tuổi trở lên cũng phải tham gia bảo hiểm y tế theo quy định của pháp luật."},
        {"role": "assistant", "content": '[{"dieu": 12, "khoan": 0, "diem": 0, "ten_vanban": "Luật Bảo hiểm y tế 2008"},{"dieu": 1, "khoan": 6, "diem": 0, "ten_vanban": "Luật Bảo hiểm y tế sửa đổi 2014"}]'}
    ]

    prompt = f"""
    Bạn là trợ lý hữu ích, hãy giúp người dùng xác định các thông tin pháp luật được nêu trong thông tin sau:
    - Điều
    - Khoản
    - Điểm
    - Tên văn bản pháp luật
    Quy tắc:
    - Đoạn văn có thể chứa nhiều văn bản pháp luật
    - <Tên văn bản pháp luật> bắt buộc phải có
    - Các thông tin còn lại có thể có đề cập hoặc không. Nếu có đề cập HÃY trả về thông tin đó, Nếu không đề cập HÃY mặc định bằng 0.

    Trả lại kết quả theo cấu trúc JSON như sau:
    [
        {{
            "dieu": <Điều>,
            "khoan": <Khoản>,
            "diem": <Điểm>,
            "ten_vanban": <Tên văn bản pháp luật>
        }},
        ...
    ]
    LƯU Ý:
    - Chỉ trả về dạng JSON, không thêm bất cứ thông tin nào.
    - Trả về một mảng các đối tượng JSON, mỗi đối tượng tương ứng với một văn bản pháp luật được đề cập.

    Thông tin của người dùng: {chunking}
    """
    
    try:
        messages = [
            {"role": "system", "content": "Bạn là một trợ lý AI hữu ích cho việc trích xuất các thông tin về văn bản luật của Việt Nam."}
        ]
        messages.extend(few_shot_examples)
        messages.append({"role": "user", "content": prompt})
        
        response = client.chat.completions.create(
            model= "gpt-4o-mini",
            messages=messages,
            max_tokens=200,
        )
        
        # Lấy nội dung phản hồi
        content = response.choices[0].message.content
        
        # Chuyển đổi chuỗi JSON thành đối tượng Python
        json_data = json.loads(content)
        
        return json_data
    except Exception as e:
        print(f"Lỗi khi xử lý chunking: {e}")
        return None

def process_text(text, model):
    try:
        result = generate_JSON_chunking(text)
        ccpls_code = []
        for ccpl in result:
            lawid = get_api_lawid(ccpl['ten_vanban'], model)
            ccpls_code.append(str(lawid) +'_'+ str(ccpl['dieu']) + '_'+ str(ccpl['khoan']) +'_'+ str(ccpl['diem']))
        return ccpls_code
    except Exception as e:
        print(f"Lỗi khi xử lý văn bản: {e}")
        return None

# Sử dụng hàm
input_text = """
Trong trường hợp doanh nghiệp tư nhân không có tư cách pháp nhân, nếu có tranh chấp hoặc kiện tụng, người khởi kiện sẽ kiện chủ doanh nghiệp tư nhân. Cụ thể, theo quy định tại Điều 190 của Luật Doanh nghiệp 2020, chủ doanh nghiệp tư nhân là người đại diện theo pháp luật cho doanh nghiệp và sẽ chịu trách nhiệm về mọi hoạt động kinh doanh của doanh nghiệp.\\n\\nDo đó, khi có tranh chấp phát sinh từ hoạt động kinh doanh của doanh nghiệp tư nhân, các bên liên quan sẽ kiện chủ doanh nghiệp tư nhân, không phải kiện doanh nghiệp tư nhân. Chủ doanh nghiệp sẽ phải đứng ra giải quyết các vấn đề pháp lý liên quan đến doanh nghiệp của mình.
"""

result = process_text(input_text, model)
print(json.dumps(result, indent=2, ensure_ascii=False))

[
  "427301_190_0_0"
]


In [12]:
def api_search_for_test(input, model_similar):
    client = OpenAI()
    gooole_request = google_search(input)
    # print(gooole_request)
    matching_result = find_max_result_google_search(input, gooole_request, model_similar)
    print(matching_result['href'])
    tes1 = crawl_data(matching_result)
    body_html = str(tes1)
    data_content = markdownify.markdownify(body_html, heading_style="ATX")
    headers_to_split_on = [
    # ("#", "Header 1"),
    ("##", "Header 2"),
    # ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    md_header_splits = markdown_splitter.split_text(data_content)
    if len(md_header_splits) < 2:
        headers_to_split_on = [("#", "Header 1"),]
        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
        md_header_splits = markdown_splitter.split_text(data_content)
    # print(len(md_header_splits))
    # print(md_header_splits)
    # print("-------------------------------")
    header_ques = []
    content_ans = []
    for temp in md_header_splits:
        temp1 = temp.metadata
        # print(temp1)
        plain_text = ' '.join(list(temp1.values())).replace('**', '')
        if plain_text != "" and ".html" not in plain_text and "[" not in plain_text and "văn bản liên quan" not in plain_text.lower() and "hỏi đáp pháp luật liên quan" not in plain_text.lower():
            header_ques.append(plain_text)
            content_ans.append(temp.page_content)
        # print(plain_text)
    # if not header_ques:
    #     headers_to_split_on = [("#", "Header 1"),]
         
    # print(content_temp[-1])
    # print(find_max_result_question(input, header_ques, model))
    answer_content = content_ans[find_max_result_question(input, header_ques, model_similar)]
    # print(header_ques[find_max_result_question(input, header_ques, model)])
    # print("-------------------------------")
    # print(answer_content)
    # print("-------------------------------")
    prompt = """
        Bạn là một chuyên gia Luật tại Việt Nam, bạn hãy dựa vào nội dung bên dưới và cung cấp cho người dùng câu trả lời chính xác, trung thực và phải luôn có trích dẫn pháp lý trong nội dung trả lời.
        """
        # Câu trả lời mở đầu bằng căn cứ theo điểm, khoản, Điều, tên văn bản nếu có...

    MODEL = "gpt-4o-mini"
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            # {"role": "system", "content": "bạn là một chuyên gia về luật. "},
            {"role": "system", "content": prompt},
            {"role": "assistant", "content": answer_content},
            {"role": "user", "content": input},
        ],
        temperature = 0.1,
    )

    # Bước 2: Phân tích cú pháp JSON
    data = json.loads(response.model_dump_json())
    token_using_output = []
    token_using = {
        "model": data['model'],
        "completion_tokens": data['usage']['completion_tokens'],
        "prompt_tokens": data['usage']['prompt_tokens']
    }
    token_using_output.append(token_using)
    answer_for_bot = data['choices'][0]['message']['content']
    # split_text[0].strip()
    output_api_search = {
        "answer": answer_for_bot,
        "ccpls": process_text(answer_for_bot, model_similar),
        # "answer": answer_for_bot,
        # "ccpls": split_text[1].strip(),
        "usage": token_using_output
    }
    # print(header_ques)
    return json.dumps(output_api_search, ensure_ascii=False)

In [13]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('keepitreal/vietnamese-sbert')

In [15]:
api_search_for_test('Doanh nghiệp FDI khi bán buôn rượu có bắt buộc phải có hợp đồng nguyên tắc của thương nhân sản xuất rượu?', model)

Rate limit or other exception: https://duckduckgo.com/ return None. params=None content=None data={'q': 'Doanh nghiệp FDI khi bán buôn rượu có bắt buộc phải có hợp đồng nguyên tắc của thương nhân sản xuất rượu? -site:https://vietnamtourism.gov.vn -site:https://thuvienphapluat.vn/van-ban -site:https://thuvienphapluat.vn/cong-dong-dan-luat -site:https://vi.wikipedia.org -site:https://careerviet.vn -site:https://lawkey.vn -site:https://chinhphu.vn/ -site:https://luatminhkhue.vn -site:https://dangkykinhdoanh.gov.vn -site:https://baomoi.com -site:https://gaadvisor.net/ -site:https://www.pace.edu.vn'}


In [ ]:
# # Mở file ở chế độ đọc
# lines = []
# processed_lines = []
# with open('data_test_api_search.txt', 'r', encoding='utf-8') as file:
#     # Duyệt từng dòng trong file
#     for line in file:
#         lines.append(line)
#         # Xử lý dòng đọc được (loại bỏ ký tự xuống dòng bằng strip())
#         processed_lines.append(api_search_for_test(line, model))


https://thienluatphat.vn/doanh-nghiep-la-gi


c:\Users\My_Pc\anaconda3\envs\Minh-AI\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thienluatphat.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://thuvienphapluat.vn/chinh-sach-phap-luat-moi/vn/ho-tro-phap-luat/tu-van-phap-luat/58090/le-phi-dang-ky-doanh-nghiep-nam-2024-la-bao-nhieu


c:\Users\My_Pc\anaconda3\envs\Minh-AI\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thuvienphapluat.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://thuvienphapluat.vn/chinh-sach-phap-luat-moi/vn/ho-tro-phap-luat/chinh-sach-moi/37296/tieu-chi-xac-dinh-doanh-nghiep-nho-va-vua-moi-nhat


...

c:\Users\My_Pc\anaconda3\envs\Minh-AI\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thuvienphapluat.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://thuvienphapluat.vn/phap-luat/quy-dinh-ve-hinh-thanh-tai-san-chung-cua-vo-chong-sau-khi-ket-hon-va-sau-khi-ly-hon-thi-tai-san-chu-92406-21320.html


c:\Users\My_Pc\anaconda3\envs\Minh-AI\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thuvienphapluat.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Lỗi khi xử lý chunking: Expecting property name enclosed in double quotes: line 27 column 1 (char 510)
Lỗi khi xử lý văn bản: 'NoneType' object is not iterable
Rate limit or other exception: https://duckduckgo.com/ return None. params=None content=None data={'q': 'Cho tôi hỏi có thể mở công ty kế toán dưới hình thức doanh nghiệp tư nhân được không?\n -site:https://vietnamtourism.gov.vn -site:https://thuvienphapluat.vn/van-ban -site:https://thuvienphapluat.vn/cong-dong-dan-luat -site:https://vi.wikipedia.org -site:https://careerviet.vn -site:https://lawkey.vn -site:https://chinhphu.vn/ -site:https://luatminhkhue.vn -site:https://dangkykinhdoanh.gov.vn -site:https://baomoi.com -site:https://gaadvisor.net/ -site:https://www.pace.edu.vn'}
Rate limit or other exception: https://duckduckgo.com/ return None. params=None content=None data={'q': 'Cho tôi hỏi có thể mở công ty kế toán dưới hình thức doanh nghiệp tư nhân được không?\n -site:https://vietnamtourism.gov.vn -site:https://thuvienphapl

KeyboardInterrupt: 

In [ ]:
# import pandas as pd
# processed_lines.append('')
# df = pd.DataFrame({
#     'Câu hỏi': lines,
#     'Câu trả lời': processed_lines
# })

# # Ghi DataFrame vào file Excel
# df.to_excel('ket_qua.xlsx', index=False)